In [1]:
import torch
import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
src_text = [
   'If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it\'s other locations in NJ and never had a bad experience. The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.'
]

In [10]:
from transformers import pipeline

summarizer = pipeline("summarization", "jordiclive/flan-t5-3b-summarizer", torch_dtype=torch.bfloat16)

raw_document = src_text[0]
prompt = "Produce an article summary of the following news article:"
results = summarizer(
        f"{prompt} {raw_document}",
        num_beams=5,
        min_length=5,
        no_repeat_ngram_size=3,
        truncation=True,
        max_length=512,
    )


Your max_length is set to 512, but your input_length is only 139. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


In [11]:
tgt_text = [result['summary_text'] for result in results]
tgt_text

['The food is good, but takes a very long time to come out . The waitstaff is very young, but usually pleasant .']

In [12]:
from calculate_metrics import compute_metrics
compute_metrics(src_text, tgt_text)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'rough': [{'rouge-1': {'r': 0.21518987341772153,
    'p': 1.0,
    'f': 0.3541666637521702},
   'rouge-2': {'r': 0.18446601941747573, 'p': 0.95, 'f': 0.30894308670764764},
   'rouge-l': {'r': 0.21518987341772153, 'p': 1.0, 'f': 0.3541666637521702}}],
 'bleu': 0,
 'bert_score': (tensor([0.9330]), tensor([0.8536]), tensor([0.8915]))}